# 中文观点抽取类情感分析

目标情感分析（Target-Based Sentiment Analysis，TBSA）是情感分析领域的一个基础问题。其目的是从一句话中识别出情感目标(或评价对象)，并预测其情感极性（积极，消极或中性）。例如，给定一句话，“重庆老灶火锅还是很赞的，有机会可以尝试一下！”模型的任务就是找出这句话描述的评价对象，并判断其情感极性。这里“重庆老灶火锅”就是评价对象（Opinion Target），它们对应的情感极性为“积极”。

一个完整的目标情感分析任务，可以分为两个子任务：观点抽取（或评价对象抽取）（Opinion Target Extraction，OTE）和目标情感分类（Target Sentiment Classification，TSC）。这里我们主要研究观点抽取这个子任务。可参考千言数据集情感分析比赛[官网](https://aistudio.baidu.com/aistudio/competition/detail/50)。


## 1 中文观点抽取任务简介

观点抽取：对于给定的文本 `d`，系统需要根据文本的内容，给出其中描述的评价对象 `a`，其中评价对象 `a` 一定在文本 `d` 中出现。数据集中每个样本是一个二元组 `<d, a>`，样例如下：
```
输入文本（d）：重庆老灶火锅还是很赞的，有机会可以尝试一下！
评价对象（a）：重庆老灶火锅
```

## 2 基于抽取式MRC框架的中文观点抽取实现

观点抽取任务的目标是从给定的文本中识别出其评价对象。目前主流的解决方案是基于序列标注的方法。序列标注采用某种编码方案（如BIO或IBEO等），对句子中的每一个词进行标注，比如我们采用 `BIO` 对“重庆老灶火锅还是很赞的，有机会可以尝试一下！”这句话进行标注：
```
tokens: ['重', '庆', '老', '灶', '火', '锅', '还', '是', '很', '赞', '的', '，', '有', '机', '会', '可', '以', '尝', '试', '一', '下', '！']
label:  ['B', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

```
然后构建序列标注模型，预测句子中的每一个词的标记。

鉴于基于指针网络的方法在抽取式机器阅读理解任务中的广泛应用，本文提出了观点抽取任务的另一种解决方案，基于抽取式 `MRC` 框架的观点抽取方法。其基本思想是将原来的序列标注任务转化为一个抽取式机器阅读理解任务去解。具体方法是，采用指针的方法对句子中的评价对象进行重新标注。观点抽取任务的目标是从一句话中抽取出这句话的评价对象，且评价对象在这句话中。我们将观点抽取任务中的句子对应到抽取式阅读理解任务中的段落，评价对象即为答案。基于抽取式 `MRC` 框架的观点抽取任务可定义为：
```
给定一个句子 `S` 和问题 `Q`，模型的目标是给出问题 `Q` 的答案 `A`，即评价对象，且答案 `A` 是出现在段落 `S` 中一段连续文本。
```
具体来说，我们需要标注出评价对象在句子中的开始位置和结束位置，如上面那个示例，评价对象“重庆老灶火锅”出现在句子中的开始位置和结束位置分别为 `0` 和 `5`。这样，我们只需要构建模型，预测评价对象出现在句子中的开始位置和结束位置，即可完成观点抽取的目标。本文实现的基于抽取式 `MRC` 框架的观点抽取模型，在测试集上取得了优于官方基于序列标注方法的 `baseline` 结果。具体地，分别在 `COTE-DP`, `COTE-BD` 和 `COTE-MFW` 三个中文观点抽取数据集的测试集上取得了 6.34%	3.45%和1.75%的提升。

本文使用的 `Pre-training + Fine-tuning` 模式的抽取式机器阅读理解架构如下，其中预训练模型使用的是 `ernie-gram-zh`：

![](https://ai-studio-static-online.cdn.bcebos.com/a479d066c6e340f2a0a28ab580dcc73393e072e51e1b462aae3aeb9cedb62c51)


### 2.1 数据处理

本文最关键的部分是，将观点抽取数据集转换成 `SQuAD` 兼容的格式。注意，这里的 `SQuAD` 兼容格式指的不是 `SQuAD` 原始的 `json` 文件格式，而是抽取式机器阅读理解统一的输入样本格式。

以 `COTE-DP`为例，`PaddleNLP` 自带的观点抽取数据集格式如下：
```
{
	'tokens': ['重', '庆', '老', '灶', '火', '锅', '还', '是', '很', '赞', '的', '，', '有', '机', '会', '可', '以', '尝', '试', '一', '下', '！'], 
    'labels': [0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 
    'entity': '重庆老灶火锅'
}
```
转换成 `SQuAD` 兼容的数据格式，即抽取式 `MRC` 模型输入样本的格式如下：
```
{
	'id': 'qid0',
    'title': '',
    'context': '重庆老灶火锅还是很赞的，有机会可以尝试一下！',
    'question': '评价对象',
    'answers': ['重庆老灶火锅'],
    'answer_starts': [0]
}
```


In [1]:
# 将 PaddleNLP 更新到最新版本
!pip install paddlenlp==2.0.8 -i https://mirror.baidu.com/pypi/simple/

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.7/571.7 kB 7.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: paddlenlp
    Found existing installation: paddlenlp 2.1.1
    Uninstalling paddlenlp-2.1.1:
      Successfully uninstalled paddlenlp-2.1.1

[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import paddlenlp
from paddlenlp.datasets import load_dataset
from paddlenlp.datasets import MapDataset

In [4]:
def create_dataset(data_name='dp', split='train'):
    """根据 data_name 和 split 参数创建数据集
    Args:
        data_name: str, 'dp', 'bd', 'mfw'
        split: str, 'train', ''test
    
    """

    # 由于 COTE 数据集只提供了训练集和测试集，所以 split 参数只能是 'train' 或 'test'
    assert isinstance(split, str), 'split must be str, it could be "train" or "test".'

    if split == 'train':
        is_test = False
    elif split == 'test':
        is_test = True
    else:
        raise ValueError('split must be "train" or "test".')

    # 根据 data_name 和 split 创建数据集 (未来会从原始数据集中重新构建 SQuAD 兼容数据集)
    dataset = load_dataset('cote', data_name, splits=[split], lazy=False)

    # 下面我们将数据集转换成 SQuAD 兼容的格式
    examples = []
    for idx, example in enumerate(dataset):
        qid = 'qid' + str(idx)
        # tokens 对应 MRC 中的 context
        context = ''.join(example['tokens'])
        # 注意，原始的样本好多是以空格或NBSP字符开头，对于基于指针的方法这类位置敏感的方法而言
        # 需要将开头的空格去掉
        context = context.strip()
        
        # 原数据集里没有 question，需要我们自己设定一个。对于观点抽取任务的问题，
        # 我们可以设为：'这句话的评价对象是什么？'
        # 这里我简单的将问题设为：'评价对象'
        # 问题的设定，对模型性能的影响，这里我没有做过多研究
        # 感兴趣可以将 question 设定为一个不相干的问题试试看，
        # 比如：'你吃过了吗？'
        question = '评价对象'  
        if not is_test:  # 训练集
            answer = example['entity']

            # 过滤掉没有答案的样本
            answer_start = context.find(answer)
            if answer_start < 0:
                continue

            new_example = {
                'id': qid,
                'title': '',
                'context': context,
                'question': question,
                'answers': [answer],
                'answer_starts': [answer_start]
            }
        else:  # 测试集   
            new_example = {
                'id': qid,
                'title':'',
                'context': context,
                'question': question,
                'answers': [],
                'answer_starts': []
            }

        examples.append(new_example)
    
    # 根据样本列表创建一个 MapDataset 对象
    dataset = MapDataset(examples)

    # 返回数据集
    return dataset

我们来看一下我们创建的数据集格式是否和抽取式 `MRC` 数据集（比如，Dureader-robust）格式一致。

In [5]:
train_cote_dp = create_dataset('dp', split='train')
train_robust = load_dataset('dureader_robust', splits='train')

100%|██████████| 20038/20038 [00:00<00:00, 42477.93it/s]


In [6]:
train_cote_dp[0]

{'id': 'qid0',
 'title': '',
 'context': '重庆老灶火锅还是很赞的，有机会可以尝试一下！',
 'question': '评价对象',
 'answers': ['重庆老灶火锅'],
 'answer_starts': [0]}

In [7]:
train_robust[0]

{'id': '0a25cb4bc1ab6f474c699884e04601e4',
 'title': '',
 'context': '第35集雪见缓缓张开眼睛，景天又惊又喜之际，长卿和紫萱的仙船驶至，见众人无恙，也十分高兴。众人登船，用尽合力把自身的真气和水分输给她。雪见终于醒过来了，但却一脸木然，全无反应。众人向常胤求助，却发现人世界竟没有雪见的身世纪录。长卿询问清微的身世，清微语带双关说一切上了天界便有答案。长卿驾驶仙船，众人决定立马动身，往天界而去。众人来到一荒山，长卿指出，魔界和天界相连。由魔界进入通过神魔之井，便可登天。众人至魔界入口，仿若一黑色的蝙蝠洞，但始终无法进入。后来花楹发现只要有翅膀便能飞入。于是景天等人打下许多乌鸦，模仿重楼的翅膀，制作数对翅膀状巨物。刚佩戴在身，便被吸入洞口。众人摔落在地，抬头发现魔界守卫。景天和众魔套交情，自称和魔尊重楼相熟，众魔不理，打了起来。',
 'question': '仙剑奇侠传3第几集上天界',
 'answers': ['第35集'],
 'answer_starts': [0]}

### 2.2 模型训练与评估

In [8]:
import json
import math
import os
import random
import time
from functools import partial

import numpy as np
import paddle
from paddle.io import DataLoader
from paddle.io import BatchSampler
from paddle.io import DistributedBatchSampler
from paddlenlp.data import Dict
from paddlenlp.data import Pad
from paddlenlp.data import Stack
from paddlenlp.data import Tuple
from paddlenlp.datasets import load_dataset
from paddlenlp.datasets import MapDataset
from paddlenlp.ops.optimizer import AdamW
from paddlenlp.transformers import BertForQuestionAnswering
from paddlenlp.transformers import BertTokenizer
from paddlenlp.transformers import ErnieForQuestionAnswering
from paddlenlp.transformers import ErnieTokenizer
from paddlenlp.transformers import ErnieGramForQuestionAnswering
from paddlenlp.transformers import ErnieGramTokenizer
from paddlenlp.transformers import RobertaForQuestionAnswering
from paddlenlp.transformers import RobertaTokenizer
from paddlenlp.transformers import LinearDecayWithWarmup

from sklearn.model_selection import train_test_split

from paddlenlp.transformers import SkepTokenizer
from models import SkepForQuestionAnswering

from config import Config
from dataset import create_dataset
from utils import CrossEntropyLossForSQuAD
from utils import evaluate
from utils import predict
from utils import prepare_train_features
from utils import prepare_validation_features
from utils import set_seed

In [9]:
MODEL_CLASSES = {
    "bert": (BertForQuestionAnswering, BertTokenizer),
    "ernie": (ErnieForQuestionAnswering, ErnieTokenizer),
    "ernie_gram": (ErnieGramForQuestionAnswering, ErnieGramTokenizer),
    "roberta": (RobertaForQuestionAnswering, RobertaTokenizer),
}

#### 2.2.1 模型训与评估练代码的封装

In [10]:
def do_train(args):
    
    paddle.set_device(args.device)
    set_seed(args)

    # 定义分词器
    args.model_type = args.model_type.lower()
    model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
    tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path)

    # 加载数据集，并划分出验证集
    train_ds = create_dataset(data_name=args.data_name, split='train')
    train_ds, dev_ds = train_test_split(train_ds, test_size=0.3, random_state=args.seed)
    train_ds, dev_ds = MapDataset(train_ds), MapDataset(dev_ds)

    # 将数据集转化为模型输入的特征
    train_trans_func = partial(
        prepare_train_features, 
        max_seq_length=args.max_seq_length, 
        doc_stride=args.doc_stride,
        tokenizer=tokenizer
    )
    train_ds.map(train_trans_func, batched=True)

    dev_trans_func = partial(
        prepare_validation_features, 
        max_seq_length=args.max_seq_length, 
        doc_stride=args.doc_stride,
        tokenizer=tokenizer
    )
    dev_ds.map(dev_trans_func, batched=True)

    # 定义BatchSampler
    train_batch_sampler = DistributedBatchSampler(
            dataset=train_ds, 
            batch_size=args.batch_size, 
            shuffle=True
    )
    dev_batch_sampler = BatchSampler(
        dataset=dev_ds, 
        batch_size=args.batch_size, 
        shuffle=False
    )
    # 定义batchify_fn
    train_batchify_fn = lambda samples, fn=Dict({
        "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id),
        "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
        "start_positions": Stack(dtype="int64"),
        "end_positions": Stack(dtype="int64")
    }): fn(samples)
    dev_batchify_fn = lambda samples, fn=Dict({
        "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id),
        "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id)
    }): fn(samples)

    # 构造DataLoader
    train_data_loader = DataLoader(
        dataset=train_ds,
        batch_sampler=train_batch_sampler,
        collate_fn=train_batchify_fn,
        return_list=True
    )
    dev_data_loader =  DataLoader(
        dataset=dev_ds,
        batch_sampler=dev_batch_sampler,
        collate_fn=dev_batchify_fn,
        return_list=True
    )

    output_dir = os.path.join(args.output_dir, 'best_model')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 创建模型
    model = model_class.from_pretrained(args.model_name_or_path)
    # model = model_class.from_pretrained(output_dir)


    num_training_steps = args.max_steps if args.max_steps > 0 else len(
        train_data_loader) * args.num_train_epochs
    num_train_epochs = math.ceil(num_training_steps / len(train_data_loader))

    num_batches = len(train_data_loader)

    # 学习率调度策略 - 带学习率预热的线性衰减
    lr_scheduler = LinearDecayWithWarmup(
        learning_rate=args.learning_rate, 
        total_steps=num_training_steps,
        warmup=args.warmup_proportion
    )

    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]

    # 定义优化器
    optimizer = paddle.optimizer.AdamW(
        learning_rate=lr_scheduler,
        epsilon=args.adam_epsilon,
        parameters=model.parameters(),
        weight_decay=args.weight_decay,
        apply_decay_param_fun=lambda x: x in decay_params
    )

    # 定义损失函数
    criterion = CrossEntropyLossForSQuAD()

    best_val_f1 = 0.0

    global_step = 0
    tic_train = time.time()

    # 模型开始训练
    for epoch in range(1, num_train_epochs + 1):
        for step, batch in enumerate(train_data_loader, start=1):

            global_step += 1
            
            input_ids, segment_ids, start_positions, end_positions = batch
            logits = model(input_ids=input_ids, token_type_ids=segment_ids)
            loss = criterion(logits, (start_positions, end_positions))

            if global_step % args.log_steps == 0 :
                # print("global step %d, epoch: %d, batch: %d/%d, loss: %.5f,  speed: %.2f step/s" % (
                #     global_step, epoch, step, num_batches, loss, args.log_steps / (time.time() - tic_train)))
                
                print("global step %d, epoch: %d, batch: %d/%d, loss: %.5f,  speed: %.2f step/s, lr: %1.16e" % (
                    global_step, epoch, step, num_batches, loss, args.log_steps / (time.time() - tic_train), lr_scheduler.get_lr()))
                
                tic_train = time.time()
            
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.clear_grad()
            
            # 每 save_steps 个迭代，做一次模型评估，并且模型训练结束后再做一次模型评估
            if global_step % args.save_steps == 0 or global_step == num_training_steps:
                em, f1 = evaluate(model=model, data_loader=dev_data_loader)

                print("global step: %d, eval dev Exact Mactch: %.5f, f1_score: %.5f" % (global_step, em, f1))
                
                # 保存 f1 值最好的模型
                if f1 > best_val_f1:
                    best_val_f1 = f1

                    print("save model at global step: %d, best eval f1_score: %.5f" % (global_step, best_val_f1))

                    model.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                if global_step == num_training_steps:
                    break


#### 2.2.2 模型训练参数的定义

In [11]:
args = Config(model_type='roberta',   # ernie_gram, skep
              model_name_or_path='roberta-wwm-ext-large',   # ernie-gram-zh skep_ernie_1.0_large_ch
              data_name='dp',  # dp, bd, mfw
              output_dir='./outputs/cote/dp/roberta',  # './outputs/cote/{data_name}/{model_type}'
              
              max_seq_length=128, 
              batch_size=32,
              learning_rate=5e-5,
              num_train_epochs=10,
              log_steps=10,          # dp, mfw == 20,  bd == 10
              save_steps=100,        # dp, mfw == 200, bd == 100
              doc_stride=64,
              warmup_proportion=0.1,
              weight_decay=0.01)

#### 2.2.2 启动训练

In [12]:
do_train(args)

[2023-02-28 10:51:50,743] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/vocab.txt and saved to /home/aistudio/.paddlenlp/models/roberta-wwm-ext-large
[2023-02-28 10:51:50,745] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/vocab.txt
100%|██████████| 107/107 [00:00<00:00, 7612.81it/s]
[2023-02-28 10:52:18,519] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/roberta_chn_large.pdparams and saved to /home/aistudio/.paddlenlp/models/roberta-wwm-ext-large
[2023-02-28 10:52:18,523] [    INFO] - Downloading roberta_chn_large.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/roberta_chn_large.pdparams
100%|██████████| 1271615/1271615 [07:29<00:00, 2830.22it/s] 
W0228 10:59:48.004133   144 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0228 

### 2.3 模型预测

#### 2.3.1 模型预测代码的封装

In [13]:
def do_predict(args):

    paddle.set_device(args.device)

    output_dir = os.path.join(args.output_dir, "best_model")

    # 加载测试集
    test_ds = create_dataset(data_name=args.data_name, split='test')

    model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
    tokenizer = tokenizer_class.from_pretrained(output_dir)

    # 将数据集转化为模型输入的特征
    test_trans_func = partial(
        prepare_validation_features, 
        max_seq_length=args.max_seq_length, 
        doc_stride=args.doc_stride,
        tokenizer=tokenizer
    )
    test_ds.map(test_trans_func, batched=True)

    # test BatchSampler
    test_batch_sampler = BatchSampler(
        dataset=test_ds, 
        batch_size=args.batch_size, 
        shuffle=False
    )

    # test dataset features batchify
    test_batchify_fn = lambda samples, fn=Dict({
        "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id),
        "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id)
    }): fn(samples)

    # test DataLoader
    test_data_loader =  DataLoader(
        dataset=test_ds,
        batch_sampler=test_batch_sampler,
        collate_fn=test_batchify_fn,
        return_list=True
    )

    model = model_class.from_pretrained(output_dir)
    
    all_predictions = predict(model, test_data_loader)

    # Can also write all_nbest_json and scores_diff_json files if needed
    with open('COTE_' + args.data_name.upper() + '.tsv', "w", encoding='utf-8') as writer:
        writer.write('index\tprediction\n')
        idx = 0
        for example in test_data_loader.dataset.data:
            writer.write(str(idx) + '\t' + all_predictions[example['id']] + '\n')
            idx += 1

    count = 0
    for example in test_data_loader.dataset.data:
        count += 1
        print()
        print('问题：',example['question'])
        print('原文：',''.join(example['context']))
        print('答案：',all_predictions[example['id']])
        if count >= 10:
            break

#### 2.3.2 启动模型预测

In [ ]:
do_predict(args)

### 2.4 实验结果

实验结果如下表所示：
||DP|BD|MFW|
|---|---|---|---|
|Official(baseline)|0.8496|0.8649|0.8732||
|ernie-gram|**0.913**|**0.8994**|**0.8907**|
|diff|**0.0634**|**0.0345**|**0.0175**|
|roberta-large|**0.9099**|**0.8917**|**0.8895**|
|diff|**0.0603**|**0.0268**|**0.0163**|

总的来看，本文提出的基于抽取 `MRC` 框架的方法在中文观点抽取的三个数据集上均取得了优于官方基于序列标注方法的 `baseline`，证明了本文方法的有效性。特别是在 `COTE-DP` 数据集上，本文提出的方法更是取得了 `6.34%` 的大幅提升。 这得益于抽取式阅读理解框架中的基于指针的标注方案。

在预训练模型的选择上，本文主要使用了 `ernie-gram` 和 `roberta-large` 两个模型，其中 `ernie-gram` 为 `base` 模型。让我们很意外的是，在相同超参配置下，`large` 模型并没有取得优于 `base` 模型的效果。我们认为这可能是 `large` 模型更难优化的缘故。这里实验使用 `large` 模型主要是想和 `baseline` 做一下对比的。而 `baseline` 使用的 `skep` 模型是一个 `large` 模型，但是由于 `skep` 并不支持抽取式阅读理解任务，本文尝试对照 `PaddleNLP` 其它抽取式阅读理解模型来实现基于 `skep` 的抽取式阅读理解模型，然而实验失败了，由于时间原因，目前还没找到原因，后期有时间会继续该实验。

综上，我们认为抽取式 `MRC` 框架是解决观点抽取任务的一个非常富有前景的解决方案。


## 3 意见反馈

关于本项目有什么问题或意见可随时在NLP打卡营的 `QQ` 群里 `@我爱志方小姐`。如果您不在 `QQ` 群里里，也欢迎您在评论区留下您宝贵的建议~

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 